<a href="https://colab.research.google.com/github/Matias-sole/ProyectoDSpartel_SOLE/blob/main/Validador_PDV_Prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np

np.random.seed(42)

n = 300

# PDVs rotos
pdvs = [f"PDV_{i}" for i in range(1, 51)]
pdv_col = np.random.choice(pdvs + ["", " ", None], size=n)

# Fechas rotas
dates = []
for _ in range(n):
    r = np.random.rand()
    if r < 0.4:
        dates.append(f"2025-0{np.random.randint(1,9)}-{np.random.randint(10,28)}")
    elif r < 0.7:
        dates.append(f"{np.random.randint(1,28)}/0{np.random.randint(1,9)}/2025")
    else:
        dates.append("fecha_erronea")

# SKUs rotos
skus_list = ["Quilmes 1L", "Quilmes Lata 473", "Stella 1L", "Corona 330", "Michelob 473",
             "Qlm_473", "STLA_1", "", None]
skus = np.random.choice(skus_list, size=n)

# Cantidad rota
cantidades = []
for _ in range(n):
    r = np.random.rand()
    if r < 0.6:
        cantidades.append(np.random.randint(1,40))
    elif r < 0.8:
        cantidades.append(str(np.random.randint(1,40)))
    else:
        cantidades.append("??")

# Latitud / Longitud rotas
lat = np.random.uniform(-34.70, -34.50, size=n).astype(object)
lon = np.random.uniform(-58.55, -58.35, size=n).astype(object)

for i in range(20):
    lat[np.random.randint(0,n)] = None
    lon[np.random.randint(0,n)] = "x"

df_broken = pd.DataFrame({
    "PDV": pdv_col,
    "Fecha": dates,
    "SKU": skus,
    "Cantidad": cantidades,
    "Latitud": lat,
    "Longitud": lon
})

# GUARDAR EN CONTENT (funciona SIEMPRE)
path = "/content/base_rota_quilmes.csv"
df_broken.to_csv(path, index=False)

path

'/content/base_rota_quilmes.csv'

In [33]:
from google.colab import files
files.download("/content/base_rota_quilmes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
import pandas as pd
import numpy as np

df = pd.read_csv("base_rota_quilmes.csv")

# Reemplaza los nombre

In [35]:
df['PDV'] = df['PDV'].astype(str).str.strip()
df['PDV'].replace({"": None, "nan": None, "None": None}, inplace=True)

/tmp/ipython-input-1229763821.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['PDV'].replace({"": None, "nan": None, "None": None}, inplace=True)


# Arregla las fechas

In [36]:
def parse_fecha(x):
    try:
        return pd.to_datetime(x, errors="coerce", dayfirst=True)
    except:
        return pd.NaT

df['Fecha'] = df['Fecha'].apply(parse_fecha)


/tmp/ipython-input-2402274786.py:3: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


# arregla los SKU

In [37]:
sku_map = {
    "Qlm_473": "Quilmes Lata 473",
    "STLA_1": "Stella 1L",
    "Michelob 473": "Michelob Ultra 473"
}

df['SKU'] = df['SKU'].replace(sku_map)
df['SKU'] = df['SKU'].fillna("SKU_sin_identificar")

In [38]:
df['Cantidad'] = pd.to_numeric(df['Cantidad'], errors="coerce")

In [39]:
df['Latitud'] = pd.to_numeric(df['Latitud'], errors="coerce")
df['Longitud'] = pd.to_numeric(df['Longitud'], errors="coerce")

# Agrega el dia de la semana dependiendo que fecha es

In [41]:
dias = {0:"Lunes",1:"Martes",2:"Miércoles",3:"Jueves",4:"Viernes",5:"Sábado",6:"Domingo"}
df['dia_semana'] = df['Fecha'].dt.dayofweek.map(dias)

# suma un stock randon

In [42]:
def prob_stock(sku):
    if "Quilmes" in sku:
        return np.random.choice([1,0], p=[0.85, 0.15])
    elif "Stella" in sku or "Corona" in sku:
        return np.random.choice([1,0], p=[0.70, 0.30])
    else:
        return np.random.choice([1,0], p=[0.60, 0.40])

df["stock"] = df["SKU"].apply(prob_stock)

# le pone un precio entre 500 y 3500

In [43]:
df["precio_correcto"] = np.random.randint(500, 3500, size=len(df))

In [44]:
df["posm"] = np.random.choice([0,1], size=len(df), p=[0.60, 0.40])

In [45]:
df["ejecucion_venta"] = np.random.choice([0,1], size=len(df), p=[0.55, 0.45])

In [46]:
df['score_salud'] = (
    df['stock']*0.45 +
    df['ejecucion_venta']*0.20 +
    df['posm']*0.15 +
    (df['precio_correcto']/3500)*0.20
)

In [47]:
df['boton_rojo'] = (df['score_salud'] < 0.4).astype(int)

In [48]:
df.to_csv("base_quilmes_validada.csv", index=False)
df.head()

,PDV,Fecha,SKU,Cantidad,Latitud,Longitud,dia_semana,stock,precio_correcto,posm,ejecucion_venta,score_salud,boton_rojo
0,PDV_39,2025-08-21,SKU_sin_identificar,NaN,-34.620235,-58.395069,Jueves,1,567,0,1,0.682400,0
1,None,2025-07-27,Corona 330,NaN,-34.583966,-58.417726,Domingo,0,2852,0,1,0.362971,1
2,PDV_29,2025-03-25,SKU_sin_identificar,NaN,-34.593279,-58.512961,Martes,0,2432,0,1,0.338971,1
3,PDV_15,NaT,SKU_sin_identificar,30.0,-34.578419,-58.515178,NaN,1,3221,0,1,0.834057,0
4,PDV_43,NaT,Michelob Ultra 473,35.0,-34.547023,-58.530321,NaN,0,864,0,0,0.049371,1


In [50]:
from google.colab import files
files.download("base_quilmes_validada.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
import os

os.listdir()

['.config',
 '.ipynb_checkpoints',
 'base_quilmes_validada.csv',
 'reporte_validador_pdv.csv',
 'base_rota_quilmes.csv',
 'sample_data']

In [53]:
df.to_csv("base_quilmes_validada.csv", index=False)

In [54]:
from google.colab import files
files.download("base_quilmes_validada.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
import os
os.listdir()

['.config',
 '.ipynb_checkpoints',
 'base_quilmes_validada.csv',
 'reporte_validador_pdv.csv',
 'base_rota_quilmes.csv',
 'sample_data']

In [56]:
df.to_csv("base_quilmes_validada.csv", index=False)

In [57]:
os.listdir()

['.config',
 '.ipynb_checkpoints',
 'base_quilmes_validada.csv',
 'reporte_validador_pdv.csv',
 'base_rota_quilmes.csv',
 'sample_data']

# aca exporta en cualquier compu

In [59]:
from google.colab import files
uploaded = files.upload()

Saving base_quilmes_validada.csv to base_quilmes_validada (1).csv


In [60]:
from google.colab import files
files.download("base_quilmes_validada.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>